轴向连接
另⼀种数据合并运算也被称作连接（concatenation）、绑定
（binding）或堆叠（stacking）。NumPy的concatenation函数
可以⽤NumPy数组来做：

In [2]:
import numpy as np
import pandas as pd
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
result = np.concatenate([arr,arr],axis=1)
result

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

对于pandas对象（如Series和DataFrame），带有标签的轴使你
能够进⼀步推⼴数组的连接运算。具体点说，你还需要考虑以下
这些东⻄：
如果对象在其它轴上的索引不同，我们应该合并这些轴的不
同元素还是只使⽤交集？
连接的数据集是否需要在结果对象中可识别？
连接轴中保存的数据是否需要保留？许多情况下，
DataFrame默认的整数标签最好在连接时删掉。
pandas的concat函数提供了⼀种能够解决这些问题的可靠⽅式。
我将给出⼀些例⼦来讲解其使⽤⽅式。假设有三个没有重叠索引
的Series

In [4]:
s1 = pd.Series([0,1],index=['a','b'])
s2 = pd.Series([2,3,4],index=['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])

In [5]:
result = pd.concat([s1,s2,s3])
result

,0
a,0
b,1
c,2
d,3
e,4
f,5
g,6


默认情况下，concat是在axis=0上⼯作的，最终产⽣⼀个新的
Series。如果传⼊axis=1，则结果就会变成⼀个
DataFrame（axis=1是列）：

In [6]:
result = pd.concat([s1,s2,s3],axis=1)
result

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，另外的轴上没有重叠，从索引的有序并集（外连
接）上就可以看出来。传⼊join='inner'即可得到它们的交集

In [7]:
s4 = pd.concat([s1,s3])
s4

,0
a,0
b,1
f,5
g,6


In [8]:
result = pd.concat([s1,s4],axis=1)
result

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [9]:
result = pd.concat([s1,s4],axis=1,join='inner')
result

,0,1
a,0,0
b,1,1


你可以通过join_axes指定要在其它轴上使⽤的索引

In [10]:
result = pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])
result

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [11]:
result = pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e'],axis=0)
result

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


不过有个问题，参与连接的⽚段在结果中区分不开。假设你想要
在连接轴上创建⼀个层次化索引。使⽤keys参数即可达到这个⽬
的：

In [12]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

如果沿着axis=1对Series进⾏合并，则keys就会成为DataFrame
的列头：

In [13]:
result = pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])
result

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


同样的逻辑也适⽤于DataFrame对象

In [14]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
df2 = pd.DataFrame(np.arange(4).reshape(2,2)+5,index=['a','c'],columns=['three','four'])

In [15]:
result = pd.concat([df1,df2],axis=1,keys=['level1','level2'])
result

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果传⼊的不是列表⽽是⼀个字典，则字典的键就会被当做keys
选项的值：

In [16]:
result = pd.concat({'level1':df1,'level2':df2},axis=1)
result

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

此外还有两个⽤于管理层次化索引创建⽅式的参数（参⻅表8
3）。举个例⼦，我们可以⽤names参数命名创建的轴级别：

In [17]:
result = pd.concat([df1,df2],axis=1,keys=['level1','level2'],names=['upper','lower'])
result

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

最后⼀个关于DataFrame的问题是，DataFrame的⾏索引不包含
任何相关数据：

In [21]:
df3 = pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df4 = pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])
df3

,a,b,c,d
0,-1.945217,-0.020205,0.197852,-1.480068
1,0.065718,-0.854256,0.990512,0.449031
2,-0.926054,1.059543,-0.576962,-0.160915


In [22]:
df4

,b,d,a
0,-2.188553,0.078121,0.622639
1,-0.512601,0.253136,1.271179


在这种情况下，传⼊ignore_index=True即可

In [23]:
result = pd.concat([df3,df4])
result

,a,b,c,d
0,-1.945217,-0.020205,0.197852,-1.480068
1,0.065718,-0.854256,0.990512,0.449031
2,-0.926054,1.059543,-0.576962,-0.160915
0,0.622639,-2.188553,NaN,0.078121
1,1.271179,-0.512601,NaN,0.253136


In [24]:
result = pd.concat([df3,df4],ignore_index=True)
result

,a,b,c,d
0,-1.945217,-0.020205,0.197852,-1.480068
1,0.065718,-0.854256,0.990512,0.449031
2,-0.926054,1.059543,-0.576962,-0.160915
3,0.622639,-2.188553,NaN,0.078121
4,1.271179,-0.512601,NaN,0.253136


合并重叠数据
还有⼀种数据组合问题不能⽤简单的合并（merge）或连接
（concatenation）运算来处理。⽐如说，你可能有索引全部或部
分重叠的两个数据集。举个有启发性的例⼦，我们使⽤NumPy
的where函数，它表示⼀种等价于⾯向数组的if-else：

np.where 的作用：

语法：np.where(condition, x, y)
condition：条件表达式（布尔值）。
x：条件为 True 时选取的值。
y：条件为 False 时选取的值。
如果 a 中的值是 NaN（pd.isnull(a) 为 True），结果取 b 中对应的值；
如果 a 中的值不是 NaN，结果取 a 中的值。

In [25]:
a = pd.Series([np.nan,2.5,0.0,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])
b = pd.Series([0.,np.nan,2.,np.nan,np.nan,5.],index=['a','b','c','d','e','f'])
b[-1] = np.nan
a

<ipython-input-25-be03d6091fdb>:3: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  b[-1] = np.nan


,0
f,NaN
e,2.5
d,0.0
c,3.5
b,4.5
a,NaN


In [26]:
b

,0
a,0.0
b,NaN
c,2.0
d,NaN
e,NaN
f,NaN


In [27]:
result = np.where(pd.isnull(a),b,a)
result

array([0. , 2.5, 0. , 3.5, 4.5, nan])

 Series有⼀个combine_first⽅法，实现的也是⼀样的功能，还带有pandas的数据对⻬：

In [30]:
result = b[:-2].combine_first(a[2:])
result

,0
a,0.0
b,4.5
c,2.0
d,0.0


对于DataFrame，combine_first⾃然也会在列上做同样的事情，
因此你可以将其看做：⽤传递对象中的数据为调⽤对象的缺失数
据“打补丁”：

In [31]:
df1 = pd.DataFrame({'a':[1.,np.nan,5.,np.nan],'b':[np.nan,2.,np.nan,6.],'c':range(2,18,4)})
df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],'b':[np.nan,3.,4.,6.,8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [32]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [33]:
result = df1.combine_first(df2)
result

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


重塑和轴向旋转

---
有许多⽤于重新排列表格型数据的基础运算。这些函数也称作重
塑（reshape）或轴向旋转（pivot）运算。
重塑层次化索引
层次化索引为DataFrame数据的重排任务提供了⼀种具有良好⼀
致性的⽅式。主要功能有⼆：
stack：将数据的列“旋转”为⾏。
unstack：将数据的⾏“旋转”为列。
我将通过⼀系列的范例来讲解这些操作。接下来看⼀个简单的
DataFrame，其中的⾏列索引均为字符串数组


In [34]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='state'),columns=pd.Index(['one','two','three'],name='numbner'))
data

numbner,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


对该数据使⽤stack⽅法即可将列转换为⾏，得到⼀个Series

In [35]:
result = data.stack()
result

state     numbner
Ohio      one        0
          two        1
          three      2
Colorado  one        3
          two        4
          three      5
dtype: int64

对于⼀个层次化索引的Series，你可以⽤unstack将其重排为⼀
个DataFrame：

In [36]:
result.unstack()

numbner,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，unstack操作的是最内层（stack也是如此）。传⼊
分层级别的编号或名称即可对其它级别进⾏unstack操作：

In [37]:
result.unstack('state')

state,Ohio,Colorado
numbner,,
one,0,3
two,1,4
three,2,5


如果不是所有的级别值都能在各分组中找到的话，则unstack操
作可能会引⼊缺失数据

In [38]:
s1 = pd.Series([0,1,2,3],index=['a','b','c','d'])
s2 = pd.Series([4,5,6],index=['c','d','e'])
data2 = pd.concat([s1,s2],keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [39]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack默认会滤除缺失数据，因此该运算是可逆的：

In [40]:
data2.unstack().stack()
# 不用滤除缺失数据的话可以 stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

在对DataFrame进⾏unstack操作时，作为旋转轴的级别将会成
为结果中的最低级别：

In [41]:
df = pd.DataFrame({'left':result,'right':result+5},columns=pd.Index(['left','right'],name='side'))
df

side              left  right
state    numbner             
Ohio     one         0      5
         two         1      6
         three       2      7
Colorado one         3      8
         two         4      9
         three       5     10

In [42]:
df.unstack('state')

side    left          right         
state   Ohio Colorado  Ohio Colorado
numbner                             
one        0        3     5        8
two        1        4     6        9
three      2        5     7       10

当调⽤stack，我们可以指明轴的名字：

In [43]:
df.unstack('state').stack('side')

<ipython-input-43-004e61fa7879>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack('state').stack('side')


state          Ohio  Colorado
numbner side                 
one     left      0         3
        right     5         8
two     left      1         4
        right     6         9
three   left      2         5
        right     7        10

将“⻓格式”旋转为“宽格式”

---
多个时间序列数据通常是以所谓的“⻓格式”（long）或“堆叠格
式”（stacked）存储在数据库和CSV中的。我们先加载⼀些示例
数据，做⼀些时间序列规整和数据清洗


In [45]:
data = pd.read_csv('macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [46]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
columns = pd.Index(['realgdp','infl','unemp'],name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D','end')
ldata = data.stack().reset_index().rename(columns={0:'value'})

<ipython-input-46-93cd14a5fb15>:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')


这就是多个时间序列（或者其它带有两个或多个键的可观察数
据，这⾥，我们的键是date和item）的⻓格式。表中的每⾏代表
⼀次观察。
关系型数据库（如MySQL）中的数据经常都是这样存储的，因
为固定架构（即列名和数据类型）有⼀个好处：随着表中数据的
添加，item列中的值的种类能够增加。在前⾯的例⼦中，date和
item通常就是主键（⽤关系型数据库的说法），不仅提供了关系
完整性，⽽且提供了更为简单的查询⽀持。有的情况下，使⽤这
样的数据会很麻烦，你可能会更喜欢DataFrame，不同的item值
分别形成⼀列，date列中的时间戳则⽤作索引。DataFrame的
pivot⽅法完全可以实现这个转换：

In [48]:
pivoted = ldata.pivot('date','item','value')
pivoted

TypeError: DataFrame.pivot() takes 1 positional argument but 4 were given

In [49]:
pivoted = ldata.pivot(index='date', columns='item', values='value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


前两个传递的值分别⽤作⾏和列索引，最后⼀个可选值则是⽤于
填充DataFrame的数据列。假设有两个需要同时重塑的数据列

In [51]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.335680
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.639185
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.186020
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.999367
4,1959-06-30 23:59:59.999999999,infl,2.340,2.324079
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.528022
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.523979
7,1959-09-30 23:59:59.999999999,infl,2.740,0.010481
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.065578
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.398173


如果忽略最后⼀个参数，得到的DataFrame就会带有层次化的
列：

In [52]:
pivoted = ldata.pivot(index='date',columns='item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.639185 -0.335680   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  2.324079  0.999367   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.010481  0.523979   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.207774  0.398173   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.765154  0.220277   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.186020  
1959-06-30 23:59:59.999999999  0.528022  
1959-09-30 23:59:59.999999999  1.065578  
1959-12-31 23:59:59.999999999 -1.098585  
1960-03-31 23:59:59.999999999 -1.699787

In [53]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


注意，pivot其实就是⽤set_index创建层次化索引，再⽤unstack
重塑：

In [54]:
unstacked = ldata.set_index(['date','item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.639185 -0.335680   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  2.324079  0.999367   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.010481  0.523979   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.207774  0.398173   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.765154  0.220277   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.443615 -1.827699   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.445221 -0.487130   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.186020  
1959-06-30 23:59:59.999999999  0.528022  
1959-09-30 23:59:59.999999999  1.065578  
1959-12-31 23:59:59.999999999 -1.098585  
1960-03-31 23:59:59.999999999 -1.699787  
1960-06-30 23:59:59.999999999  0.896387  
1960-09-30 23:59:59.999999999 -0.726338

将“宽格式”旋转为“⻓格式”

---


旋转DataFrame的逆运算是pandas.melt。它不是将⼀列转换到
多个新的DataFrame，⽽是合并多个列成为⼀个，产⽣⼀个⽐输
⼊⻓的DataFrame。看⼀个例⼦：

In [55]:
df = pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


key列可能是分组指标，其它的列是数据值。当使⽤
pandas.melt，我们必须指明哪些列是分组指标。下⾯使⽤key作
为唯⼀的分组指标：

In [56]:
melted = pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使⽤pivot，可以重塑回原来的样⼦：

In [58]:
reshaped = melted.pivot(index='key',columns='variable',values='value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


因为pivot的结果从列创建了⼀个索引，⽤作⾏标签，我们可以使
⽤reset_index将数据移回列

In [59]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


你还可以指定列的⼦集，作为值的列：

In [61]:
result = pd.melt(df,id_vars=['key'],value_vars=['A','B','C'])
result

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


 pandas.melt也可以不⽤分组指标

In [62]:
result = pd.melt(df,value_vars=['A','B','C'])
result

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [63]:
result = pd.melt(df,value_vars=['key','A','B'])
result

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
